In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
from keras.models import load_model
from keras import backend as K
from create_gif import BaseAgent
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam, SGD

In [26]:
# Start virtual display
# pip install PyVirtualDisplay
# sudo apt-get install xvfb

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1024, 768))
display.start()
import os
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

In [27]:

class ReinforceAgent(BaseAgent):
    # def __init__(self):
    def actor_critic_loss_continuous(self, advantage):
        def loss(y_true, y_pred):
            var = K.square(self.noise)
            print(var)
            denom = K.sqrt(2.0 * np.pi * var)
            prob_num = K.exp(- K.square(y_true - y_pred) / (2.0 * var))
            prob = prob_num/denom
            return -K.mean(prob * advantage)
        return loss
    
    def get_policy_model(self, lr=0.001, hidden_layer_neurons = 128, input_shape=[4], output_shape=2):
        state_input = Input(shape=input_shape)
        advantage = Input(shape=(1,))
        x = Dense(hidden_layer_neurons, activation='relu')(state_input)
        out_actions = Dense(output_shape, activation='tanh')(x)
        
        model = Model(inputs=[state_input, advantage], outputs=[out_actions])
        model.compile(Adam(lr), loss=self.actor_critic_loss_continuous(advantage))
        return model
    
    def get_action(self, eval=False):
        dummy_adv = np.zeros((1, 1))
        p = self.model.predict([self.observation.reshape(1, self.nS), dummy_adv])
        if eval is False:
            action = action_one_hot = p[0] + np.random.normal(loc=0, scale=self.noise, size=p[0].shape)
            # action = action_one_hot = np.clip(action, self.env.action_space.low, self.env.action_space.high)
        else:
            action = action_one_hot = p[0]
        return action, action_one_hot, p
    
    
    def get_entropy(self, preds, epsilon=1e-12):
        # entropy = np.mean(-np.sum(np.log(preds+epsilon)*preds, axis=1)/np.log(self.nA))
        return 1
    
    
    def get_discounted_rewards(self, r):
        # Por si es una lista
        r = np.array(r, dtype=float)
        """Take 1D float array of rewards and compute discounted reward """
        discounted_r = np.zeros_like(r)
        running_add = 0
        for t in reversed(range(0, r.size)):
            running_add = running_add * self.gamma + r[t]
            discounted_r[t] = running_add
        return discounted_r 

In [29]:
# agent = ReinforceAgent('CartPole-v1')
agent = ReinforceAgent('MountainCarContinuous-v0')
# agent.model = load_model('logs/CartPole-v1/REINFORCE_CAUSAL/1_1_0.999_0.001_1574907638.hdf5', compile=False)
# agent.model = load_model('logs/CartPole-v1/REINFORCE_BASELINE/1_1_0.999_0.001_1574904904.hdf5', compile=False)
# agent.model = load_model('logs/CartPole-v1/REINFORCE/1_1_0.999_0.001_1574901446.hdf5', compile=False)
# agent.model = load_model('logs/CartPole-v1/REINFORCE/5_1_0.999_0.001_1574902916.hdf5', compile=False)
#  agent.model = load_model('logs/Acrobot-v1/REINFORCE_CAUSAL/1_1_0.999_0.001_1574967139.hdf5', compile=False)
agent.model = load_model('./MountainCarContinuous.hdf5', compile=False)

Tensor("loss_5/dense_14_loss/Square:0", shape=(), dtype=float32)


In [30]:
_ = agent.get_eval_episode('MountainCarContinuous.gif', fps=10)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 2 arrays: [array([[-0.50939549,  0.        ]]), array([[0.]])]...

![gif](Acrobot-v1.gif)

![gif](REINFORCE_1_1_0.999_0.001_1574901446.gif)

![gif](REINFORCE_CAUSAL_1_1_0.999_0.001_1574907638.gif)

![gif](REINFORCE_BASELINE_1_1_0.999_0.001_1574904904.gif)